In [1]:
import requests
import pandas as pd
import tqdm
from dotenv import load_dotenv
import os

In [2]:
load_dotenv(dotenv_path='credentials.env')

api_key = os.getenv("API_KEY")
username = os.getenv("USER")

In [3]:
#url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/crear_usuario/?mail={username}'

In [4]:
#res = requests.post(url)
#res.json()

In [3]:
creditos_hist = pd.read_parquet('./input/creditos_hist.parquet')

In [4]:
url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/consulta_constancia_masiva/?usuario={username}&api_key={api_key}'

In [5]:
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

In [6]:
def resultados_constancias(response_json):
    datos = []
    for res in response_json['resultado']:
        datos_generales = res.get('datosGenerales') if res.get('datosGenerales') else None
        domicilio_fiscal = datos_generales.get('domicilioFiscal') if datos_generales else None
        codigo_postal = domicilio_fiscal.get('codPostal') if domicilio_fiscal else None
        provincia = domicilio_fiscal.get('descripcionProvincia') if domicilio_fiscal else None
        direccion = domicilio_fiscal.get('direccion') if domicilio_fiscal else None
        localidad = domicilio_fiscal.get('localidad') if domicilio_fiscal else None
        cuit = datos_generales.get('idPersona') if datos_generales else None
        mes_cierre = datos_generales.get('mesCierre') if datos_generales else None
        razon_social = datos_generales.get('razonSocial') if datos_generales else None
        datos_regimen = res.get('datosRegimenGeneral') if res.get('datosRegimenGeneral') else None
        datos_actividad = datos_regimen.get('actividad', []) if datos_regimen else None
        impuestos = datos_regimen.get('impuesto', []) if datos_regimen else None
        regimenes = datos_regimen.get('regimen', []) if datos_regimen else None
        
        fila = {
			'codPostal': codigo_postal,
			'identificacion': cuit,
			'provincia': provincia,
			'direccion': direccion,
			'localidad': localidad,
			'mesCierre': mes_cierre,
			'razonSocial': razon_social,
		}
        if impuestos:
            for impuesto in impuestos:
                imp_descripcion = impuesto.get('descripcionImpuesto')
                if imp_descripcion:
                    fila[imp_descripcion] = True
        
        if regimenes:
            for regimen in regimenes:
                reg_descripcion = regimen.get('descripcionRegimen')
                tipo_regimen = regimen.get('tipoRegimen')
                if reg_descripcion:
                    fila[reg_descripcion] = tipo_regimen if tipo_regimen else True
        
        if datos_actividad:
            for act in datos_actividad:
                actividad = act.get('descripcionActividad') if act.get('descripcionActividad') else None
                id_act = act.get('idActividad') if act.get('idActividad') else None
                fila_actividad = fila.copy() 
                fila_actividad['actividad'] = actividad
                fila_actividad['idActividad'] = id_act
                datos.append(fila_actividad)
    return datos

In [11]:
identificaciones = creditos_hist['identificacion'].unique()
tramos = [identificaciones[i:i + 250] for i in range(0, len(identificaciones), 250)]

In [ ]:
todos_los_datos = []
session = requests.Session()

for tramo in tqdm.tqdm(tramos):
    try:
        payload = {
        	"cuits": [str(cuit) for cuit in tramo]
        }
        response = session.post(url, json=payload, headers=headers)
        response.raise_for_status()
        datos = resultados_constancias(response.json())
        todos_los_datos.extend(datos)
    except Exception as e:
        print(e)

In [13]:
datos = pd.DataFrame(todos_los_datos)

In [17]:
datos.to_parquet('./input/constancia_inscripcion.parquet', compression= 'gzip')

In [18]:
faltan = creditos_hist.loc[~creditos_hist['identificacion'].isin(datos['identificacion'].unique())]['identificacion'].unique()

In [15]:
session = requests.Session()
prueba = []
for cuit in faltan[:10]:
    try:
        url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/consulta_constancia/?cuit={cuit}&usuario={username}&api_key={api_key}'
        response = session.get(url, headers= headers)
        response.raise_for_status()
        prueba.append(response.json())
    except Exception as e:
        print(e) 